## Archivo de creacion del mapa: FILTRO CATEGORIA PROYECTO

### 1_ Preparamos el entorno

**Importamos librerias**

In [1]:
# Preliminares
import re
import warnings

warnings.filterwarnings('ignore')

from math import radians

import folium
import matplotlib.cm as cm
import numpy as np
import openpyxl
import pandas as pd
from folium import CustomIcon
from folium.plugins import HeatMap, MarkerCluster
from matplotlib.colors import to_hex
from sklearn.metrics.pairwise import haversine_distances

**Cargamos los set**

In [2]:
# Dataframe Camiones estacionados
estacionados_camion = pd.read_excel("./estacionados_camion.xlsx", engine="openpyxl")

# Dataframe clientes 
ubi_cliente = pd.read_excel("./ubi_cliente.xlsx", engine="openpyxl")

# Tareas.xlsx
df_tareas = pd.read_excel('./Tareas-limpio.xlsx', engine="openpyxl")

**Cargamos ubicaciones extra Abitab,Red pagos, Empresa, Casas**

In [3]:
# Abitab y red pagos
lugares_pago = {
        'CODIGO': [200000, 200002, 200003, 200005, 200006, 200007, 200008, 200009, 200010, 200011, 200012, 200013, 200014, 200015, 200016, 200017, 200018, 200019, 200020, 200021, 200022, 200023, 200024, 200025, 200026, 200027, 200028, 200029, 200030, 200031],
        'NOMCLI': ['Abitab', 'ABITAB 12/18', 'Abitab', 'Abitab Ruta 87', 'Abitab 12-21', 'Abitab', 'Abitab', 'Cambilex', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab Agencia 18/09', 'Redpagos Minimercado Nico', 'Red Pagos El Dorado, Las Toscas', 'Redpagos Bachino', 'Redpagos San Luis', 'Redpagos La Tuna', 'Redpagos Provision San Jorge', 'Red Pagos', 'RedPagos', 'RedPagos', 'Cambilex', 'Redpagos Salinas', 'Redpagos Salinas Norte', 'Redpagos Pinamar', 'Redpagos Agencias Migues', 'Redpagos Saucedo'],
        'LATITUD': [-34.7773749, -34.778762, -34.7769439, -34.7722784, -34.7717067, -34.7576624, -34.7714756, -34.772715, -34.7554596, -34.7673311, -34.7506218, -34.758673, -34.769283, -34.7701505, -34.6003052, -34.79286, -34.7710358, -34.7592078, -34.7663154, -34.7801148, -34.6832898, -34.7499177, -34.7741445, -34.7679195, -34.7773725, -34.7758541, -34.77187, -34.778569, -34.4884583, -34.5999389],
        'LONGITUD': [-55.8391627, -55.8620459, -55.8489382, -55.8413815, -55.8195923, -55.766793, -55.7620991, -55.7637416, -55.7246934, -55.7220688, -55.7094748, -55.677871, -55.6553415, -55.5763933, -55.4663981, -55.49211, -55.7337499, -55.6766295, -55.5899085, -55.5607396, -55.7024884, -55.7083951, -55.7615713, -55.767487, -55.8391636, -55.8397048, -55.84086, -55.862882, -55.630918, -55.4665898]
    }

# Crear DataFrame
lugares_pago_df = pd.DataFrame(lugares_pago)

# ___________________________________________________________________________

# Añadir lugares empresa **Codigo 200.000 a 200032**
ubi_empresa = {
        'CODIGO': [100001, 100002, 100003, 100004, 100005,100000,],
        'NOMCLI': ['CASA DARIO', 'CASA HUGO', 'CASA CONRADO', 'CASA MIGUEL', 'CASA MARTIN','EMPRESA'],
        'LATITUD': [-34.770962, -34.76416, -34.766187, -34.787831, -34.761527,-34.771148],
        'LONGITUD': [-55.742166, -55.86347, -55.780495, -55.850621, -55.74736, -55.758]
    }
df_ubi_empresa = pd.DataFrame(ubi_empresa)

# Coordenadas de "EMPRESA"
coords_empresa = [-34.771148, -55.758]

# Umbral de distancia para considerar que un camión está en "EMPRESA"
umbral_distancia = 0.00001


Iconos especiales camiones numero_placa


In [4]:
# Diccionario de iconos personalizados (RUTA)
iconos_camion = {
    'AAW4251': '../num_placa_PNG/AAW4251.png',
    'BERLINGO7008': '../num_placa_PNG/BERLINGO7008.png',
    'BYD1006': '../num_placa_PNG/BYD1006.png',
    'CAA1076': '../num_placa_PNG/CAA1076.png'
}

# Normalizar las claves del diccionario de iconos
iconos_camion_normalizado = {key.replace(' ', '').upper(): value for key, value in iconos_camion.items()}

In [5]:
# Definir los colores personalizados para cada categoría
colores_categoria = {
    'Clientes Especiales': 'yellow',
    'Visitas Periódicas': 'green',  # Recomendado
    'Rutas': 'blue',
    'Creditos': 'red',
    'Otros': 'gray'  # Recomendado
}

# Definir los íconos Font Awesome para cada categoría
iconos_categoria = {
    'Clientes Especiales': 'fa-star',
    'Visitas Periódicas': 'fa-repeat',
    'Rutas': 'fa-map-marker',
    'Creditos': 'fa-credit-card',
    'Otros': 'fa-info-circle'
}

In [6]:
# Coordenadas de "EMPRESA"
coords_empresa = [-34.771148, -55.758]

# Umbral de distancia para considerar que un camión está en "EMPRESA"
umbral_distancia = 0.00001

In [7]:
# Cálculo de distancias (haversine)
def calcular_distancia(lat1, lon1, lat2, lon2):
    return haversine_distances([[radians(lat1), radians(lon1)], [radians(lat2), radians(lon2)]])[0][1]

In [8]:
# Crear el mapeo de codigo a categoria
codigo_categoria_map = dict(zip(df_tareas['CODIGO'], df_tareas['CATEGORIA']))

print("Mapeo de CODIGO a CATEGORIA:")
for codigo, categoria in codigo_categoria_map.items():
    print(f"Código: {codigo}, Categoría: {categoria}")

Mapeo de CODIGO a CATEGORIA:
Código: 12239, Categoría: Clientes Especiales
Código: 62005, Categoría: Clientes Especiales
Código: 77778, Categoría: Clientes Especiales
Código: 12869, Categoría: Clientes Especiales
Código: 22024, Categoría: Clientes Especiales
Código: 12450, Categoría: Clientes Especiales
Código: 44444, Categoría: Clientes Especiales
Código: 55555, Categoría: Clientes Especiales
Código: 11545, Categoría: Clientes Especiales
Código: 11891, Categoría: Clientes Especiales
Código: 41111, Categoría: Clientes Especiales
Código: 12904, Categoría: Clientes Especiales
Código: 12500, Categoría: Clientes Especiales
Código: 12920, Categoría: Clientes Especiales
Código: 21955, Categoría: Clientes Especiales
Código: 12286, Categoría: Clientes Especiales
Código: 12674, Categoría: Clientes Especiales
Código: 11881, Categoría: Clientes Especiales
Código: 21412, Categoría: Clientes Especiales
Código: 12284, Categoría: Clientes Especiales
Código: 11787, Categoría: Clientes Especiales
Códig

___

### 2_ Ajustes previos al mapeo

In [9]:
# Obtener semanas únicas y placas únicas
semanas = estacionados_camion['Semana'].unique()
unique_plates = estacionados_camion['Numero_de_placa'].unique()
# Normalizar las patentes en el DataFrame
estacionados_camion['Numero_de_placa'] = estacionados_camion['Numero_de_placa'].str.replace(' ', '').str.upper()

# Columna semana
def agregar_columna_semana(estacionados_camion):
    estacionados_camion['Semana'] = estacionados_camion['Tiempo_de_Inicio'].dt.strftime('%U')
  
# 4. Definir los límites del mapa
def filtrar_por_semana(estacionados_camion, semana):
    return estacionados_camion[estacionados_camion['Semana'] == semana]

  

In [10]:

# 6. Calcular clientes visitados
def obtener_clientes_visitados(camiones_placa, ubi_cliente_con_extras):
    clientes_visitados = set()
    for _, camion in camiones_placa.iterrows():
        distancias = ubi_cliente_con_extras.apply(lambda row: calcular_distancia(
            camion['camion_x'], camion['camion_y'], row['LATITUD'], row['LONGITUD']), axis=1)
        cliente_cercano_idx = distancias.idxmin()
        cliente_cercano = ubi_cliente_con_extras.loc[cliente_cercano_idx]
        clientes_visitados.add(cliente_cercano['CODIGO'])
    return clientes_visitados


In [11]:


# Ahora unimos los extras a ubi_cliente asegurando de que no se agreguen duplicados y que no se pierden los datos de ubi_cliente
ubi_cliente_con_extras = pd.concat([ubi_cliente, df_ubi_empresa, lugares_pago_df], ignore_index=True).drop_duplicates()

# Añadir la columna 'Categoria' al DataFrame 'ubi_cliente_con_extras'
# Mapear los códigos a sus respectivas categorías, si un código no está en el mapeo, se asigna 'Otros'
ubi_cliente_con_extras['Categoria'] = ubi_cliente_con_extras['CODIGO'].map(codigo_categoria_map).fillna('Otros')


# Verificar que no haya valores NaN en la columna 'Categoria'
assert ubi_cliente_con_extras['Categoria'].isna().sum() == 0, "Hay NaNs en la columna 'Categoria'"


# Definir los límites del mapa
min_lon, max_lon = ubi_cliente_con_extras['LONGITUD'].min(), ubi_cliente_con_extras['LONGITUD'].max()
min_lat, max_lat = ubi_cliente_con_extras['LATITUD'].min(), ubi_cliente_con_extras['LATITUD'].max()

In [12]:
# DataFrame de ejemplo para mapear números de placa a códigos de clientes
placas_a_codigos = pd.DataFrame({
    'Numero_de_placa': ['ABC123', 'DEF456', 'GHI789', 'JKL012', 'MNO345'],
    'CODIGO': [12239, 62005, 77778, 12869, 22024]
})

# Agregar la categoría a 'estacionados_camion' usando 'placas_a_codigos' y 'codigo_categoria_map'
estacionados_camion = estacionados_camion.merge(placas_a_codigos, on='Numero_de_placa', how='left')
estacionados_camion['Categoria'] = estacionados_camion['CODIGO'].map(codigo_categoria_map).fillna('Otros')

# Confirmar que la columna 'Categoria' está presente
print(estacionados_camion.columns)

# Categorías únicas
categorias = estacionados_camion['Categoria'].unique()

print("Categorías únicas:")
print(categorias)


Index(['Indice', 'Numero_de_placa', 'Estado_de_viaje', 'Tiempo_de_Inicio',
       'Tiempo_Final', 'Duracion', 'Lugar_de_inicio', 'camion_x', 'camion_y',
       'Semana', 'Semana_del_mes', 'CODIGO', 'Categoria'],
      dtype='object')
Categorías únicas:
['Visitas Periódicas']


### 

In [13]:
# Recrear codigo_categoria_map
codigo_categoria_map = dict(zip(df_tareas['CODIGO'], df_tareas['CATEGORIA']))


# Definir una función para obtener la categoría con un valor por defecto
def obtener_categoria(codigo, default='Otros'):
    return codigo_categoria_map.get(codigo, default)

# Añadir la columna 'Categoria' al DataFrame 'ubi_cliente_con_extras'
ubi_cliente_con_extras['Categoria'] = ubi_cliente_con_extras['CODIGO'].map(codigo_categoria_map)


print("Muestra de codigo_categoria_map:")
for codigo, categoria in list(codigo_categoria_map.items())[:5]:
    print(f"Código: {codigo}, Categoría: {categoria}")


Muestra de codigo_categoria_map:
Código: 12239, Categoría: Clientes Especiales
Código: 62005, Categoría: Clientes Especiales
Código: 77778, Categoría: Clientes Especiales
Código: 12869, Categoría: Clientes Especiales
Código: 22024, Categoría: Clientes Especiales


In [14]:
ubi_cliente_con_extras['CODIGO'] = ubi_cliente_con_extras['CODIGO'].astype(str)
df_tareas['CODIGO'] = df_tareas['CODIGO'].astype(str)

codigos_ubi = set(ubi_cliente_con_extras['CODIGO'])
codigos_tareas = set(df_tareas['CODIGO'])
interseccion = codigos_ubi.intersection(codigos_tareas)
print("Número de códigos que coinciden:", len(interseccion))
print("Ejemplos de códigos que coinciden:", list(interseccion)[:10])

Número de códigos que coinciden: 388
Ejemplos de códigos que coinciden: ['42905', '21391', '81127', '13065', '21418', '61494', '12910', '12901', '12925', '61833']


# Creamos el mapa

### Mapa 1: crea los iconos de los proyectos pero sin ningun filtro

In [ ]:
# __________________________________ Creación del Mapa Base
for semana in semanas:
    datos_semana = estacionados_camion[estacionados_camion['Semana'] == semana]
    
    # Calcular percentiles para la semana específica
    duracion_25 = datos_semana['Duracion'].quantile(0.25)
    duracion_75 = datos_semana['Duracion'].quantile(0.75)
    
    # Crear el mapa base
    mapa2 = folium.Map(
        location=[datos_semana['camion_x'].mean(), datos_semana['camion_y'].mean()],
        zoom_start=12,
        max_bounds=True,
        min_lat=min_lat,
        max_lat=max_lat,
        min_lon=min_lon,
        max_lon=max_lon
    )
    
    # __________________________________ Crear capas para cada número de placa
    for placa in unique_plates:
        capa_placa = folium.FeatureGroup(name=f"Placa {placa}", show=False)
        camiones_placa = datos_semana[datos_semana['Numero_de_placa'] == placa]
        clientes_visitados = set()
        
        # __________________________________ Agregar marcadores para cada camión de la placa
        for _, camion in camiones_placa.iterrows():
            patente = camion['Numero_de_placa']
            
            # Calcular la distancia del camión a "EMPRESA"
            distancia_a_empresa = calcular_distancia(
                camion['camion_x'], camion['camion_y'], 
                coords_empresa[0], coords_empresa[1]
            )
            
            # Si la distancia es menor que el umbral, excluir este camión
            if distancia_a_empresa < umbral_distancia:
                continue
            
            if patente in iconos_camion_normalizado:
                icon_path = iconos_camion_normalizado[patente]
                icono = folium.CustomIcon(icon_image=icon_path, icon_size=(30, 30))
                
                folium.Marker(
                    location=[camion['camion_x'], camion['camion_y']],
                    popup=f"<b>Camión: {patente}</b><br>Tiempo de Inicio: {camion['Tiempo_de_Inicio']}<br>Duración: {camion['Duracion']}<br>Tiempo Final: {camion['Tiempo_Final']}",
                    tooltip=f"Camión: {patente}",
                    icon=icono
                ).add_to(capa_placa)
            else:
                print(f"Patente no encontrada en el diccionario: {patente}")
            
            # Determinar el color del círculo en función de los percentiles
            if camion['Duracion'] <= duracion_25:
                color_circulo = 'green'
            elif camion['Duracion'] <= duracion_75:
                color_circulo = 'yellow'
            else:
                color_circulo = 'red'

            folium.Circle(
                location=[camion['camion_x'], camion['camion_y']],
                radius=50,
                color=color_circulo,
                fill=True,
                fill_color=color_circulo,
                fill_opacity=0.6,
                weight=2
            ).add_to(capa_placa)
            
            # Encontrar el cliente más cercano y marcarlo como visitado
            distancias = ubi_cliente_con_extras.apply(lambda row: calcular_distancia(
                camion['camion_x'], camion['camion_y'], row['LATITUD'], row['LONGITUD']), axis=1)
            cliente_cercano_idx = distancias.idxmin()
            cliente_cercano = ubi_cliente_con_extras.loc[cliente_cercano_idx]
            clientes_visitados.add(cliente_cercano['CODIGO'])
            
            folium.PolyLine(
                locations=[[camion['camion_x'], camion['camion_y']], 
                           [cliente_cercano['LATITUD'], cliente_cercano['LONGITUD']]],
                color="blue",
                weight=2,
                opacity=0.8
            ).add_to(capa_placa)
        
        for _, cliente in ubi_cliente_con_extras.iterrows():
            if cliente['CODIGO'] in clientes_visitados:
                categoria = codigo_categoria_map.get(str(cliente['CODIGO']), 'Otros')
                icono = iconos_categoria.get(categoria, 'fa-info-circle')
                print(f"Cliente: {cliente['CODIGO']}, Categoría: {categoria}, Icono: {icono}")
        # ... (resto del código para crear el marcador)
                print(f"Cliente: {cliente['CODIGO']}, Categoría: {categoria}, Icono: {icono}")
                print(f"Proyecto original: {df_tareas[df_tareas['CODIGO'] == cliente['CODIGO']]['PROYECTO'].values}")
                
                folium.Marker(
                    location=[cliente['LATITUD'], cliente['LONGITUD']],
                    popup=f"<b>Cliente: {cliente['CODIGO']}</b><br>Nombre: {cliente['NOMCLI']}<br>Categoría: {categoria}",
                    tooltip=f"CODIGO: {cliente['CODIGO']}<br>Nombre: {cliente['NOMCLI']}",
                    icon=folium.Icon(color='red', icon=icono, prefix='fa')
                ).add_to(capa_placa)
                        
                # Añadir capa al mapa
                capa_placa.add_to(mapa2)
        
    # Añadir ubicaciones de la empresa
    for _, empresa in df_ubi_empresa.iterrows():
        folium.Circle(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            radius=100,
            color="blue",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa2)
        
        folium.Marker(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            popup=f"<b>Empresa: {empresa['NOMCLI']}</b>",
            tooltip=f"CODIGO: {empresa['CODIGO']}<br>Nombre: {empresa['NOMCLI']}",
            icon=folium.Icon(color='blue', icon='fa-truck', prefix='fa')
        ).add_to(mapa2)

    # Añadir nuevos lugares de pago
    for _, lugar_pago in lugares_pago_df.iterrows():
        folium.Circle(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            radius=100,
            color="green",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa2)
        
        folium.Marker(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            popup=f"<b>Cliente: {lugar_pago['NOMCLI']}</b>",
            tooltip=lugar_pago['NOMCLI'],
            icon=folium.Icon(color="green", icon="dollar-sign", prefix='fa')
        ).add_to(mapa2)
        
    # Añadir capa de camiones al mapa
    folium.LayerControl().add_to(mapa2)
    
    # Guardar el mapa como archivo HTML
    mapa2.save(f"../mapas_html/mapa_semana_{semana}_proyecto.html")